# Stance Detection for the Fake News Challenge

## Identifying Textual Relationships with Deep Neural Nets

### Check the problem context [here](https://drive.google.com/open?id=1KfWaZyQdGBw8AUTacJ2yY86Yxgw2Xwq0).

### Download files required for the project from [here](https://drive.google.com/open?id=10yf39ifEwVihw4xeJJR60oeFBY30Y5J8).

## Step1: Load the given dataset  

1. Mount the google drive

2. Import Glove embeddings

3. Import the test and train datasets

### Mount the google drive to access required project files

Run the below commands

In [0]:
from google.colab import drive

In [28]:
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


#### Path for Project files on google drive

**Note:** You need to change this path according where you have kept the files in google drive. 

In [0]:
project_path = "/content/drive/My Drive/Great Learning/Residency 11/Assignment/Fake News Challenge/"

### Loading the Glove Embeddings

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'glove.6B.zip', 'r') as z:
  z.extractall()

In [0]:
#import all necessary libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, GRU, Dense
from keras.preprocessing.text import Tokenizer as tokenizer
import math
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

# Load the dataset [5 Marks]

1. Using [read_csv()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) in pandas load the given train datasets files **`train_bodies.csv`** and **`train_stances.csv`**

2. Using [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) command in pandas merge the two datasets based on the Body ID. 

Note: Save the final merged dataset in a dataframe with name **`dataset`**.

In [0]:
import os

In [67]:
pwd

'/content/drive/My Drive/Great Learning/Residency 11/Assignment/Fake News Challenge'

In [0]:
os.chdir('/content/drive/My Drive/Great Learning/Residency 11/Assignment/Fake News Challenge/')

In [69]:
pwd

'/content/drive/My Drive/Great Learning/Residency 11/Assignment/Fake News Challenge'

In [0]:
train_body_df = pd.read_csv('train_bodies.csv')
train_stances_df = pd.read_csv('train_stances.csv') 

In [71]:
train_body_df.head(5)

,Body ID,articleBody
0,0,A small meteorite crashed into a wooded area i...
1,4,Last week we hinted at what was to come as Ebo...
2,5,(NEWSER) – Wonder how long a Quarter Pounder w...
3,6,"Posting photos of a gun-toting child online, I..."
4,7,At least 25 suspected Boko Haram insurgents we...


In [72]:
train_stances_df.head(5)

,Headline,Body ID,Stance
0,Police find mass graves with at least '15 bodi...,712,unrelated
1,Hundreds of Palestinians flee floods in Gaza a...,158,agree
2,"Christian Bale passes on role of Steve Jobs, a...",137,unrelated
3,HBO and Apple in Talks for $15/Month Apple TV ...,1034,unrelated
4,Spider burrowed through tourist's stomach and ...,1923,disagree


In [0]:
dataset = train_body_df.merge(train_stances_df, how='inner', on='Body ID', copy=True)


<h2> Check1:</h2>
  
<h3> You should see the below output if you run `dataset.head()` command as given below </h3>

In [74]:
dataset.head()

,Body ID,articleBody,Headline,Stance
0,0,A small meteorite crashed into a wooded area i...,"Soldier shot, Parliament locked down after gun...",unrelated
1,0,A small meteorite crashed into a wooded area i...,Tourist dubbed ‘Spider Man’ after spider burro...,unrelated
2,0,A small meteorite crashed into a wooded area i...,Luke Somers 'killed in failed rescue attempt i...,unrelated
3,0,A small meteorite crashed into a wooded area i...,BREAKING: Soldier shot at War Memorial in Ottawa,unrelated
4,0,A small meteorite crashed into a wooded area i...,Giant 8ft 9in catfish weighing 19 stone caught...,unrelated


## Step2: Data Pre-processing and setting some hyper parameters needed for model


#### Run the code given below to set the required parameters.

1. `MAX_SENTS` = Maximum no.of sentences to consider in an article.

2. `MAX_SENT_LENGTH` = Maximum no.of words to consider in a sentence.

3. `MAX_NB_WORDS` = Maximum no.of words in the total vocabualry.

4. `MAX_SENTS_HEADING` = Maximum no.of sentences to consider in a heading of an article.

In [0]:
MAX_NB_WORDS = 20000
MAX_SENTS = 20
MAX_SENTS_HEADING = 1
MAX_SENT_LENGTH = 20
VALIDATION_SPLIT = 0.2

### Download the `Punkt` from nltk using the commands given below. This is for sentence tokenization.

For more info on how to use it, read [this](https://stackoverflow.com/questions/35275001/use-of-punktsentencetokenizer-in-nltk).



In [76]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Tokenizing the text and loading the pre-trained Glove word embeddings for each token  [5 marks] 

Keras provides [Tokenizer API](https://keras.io/preprocessing/text/) for preparing text. Read it before going any further.

#### Import the Tokenizer from keras preprocessing text

In [0]:
from keras.preprocessing.text import Tokenizer as tokenizer

#### Initialize the Tokenizer class with maximum vocabulary count as `MAX_NB_WORDS` initialized at the start of step2. 

In [0]:
t = tokenizer(num_words=MAX_NB_WORDS,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True, split=' ')

#### Now, using fit_on_texts() from Tokenizer class, lets encode the data 

Note: We need to fit articleBody and Headline also to cover all the words.

In [0]:
t.fit_on_texts(dataset['Headline'])
t.fit_on_texts(dataset['articleBody'])

#### fit_on_texts() gives the following attributes in the output as given [here](https://faroit.github.io/keras-docs/1.2.2/preprocessing/text/).

* **word_counts:** dictionary mapping words (str) to the number of times they appeared on during fit. Only set after fit_on_texts was called.

* **word_docs:** dictionary mapping words (str) to the number of documents/texts they appeared on during fit. Only set after fit_on_texts was called.

* **word_index:** dictionary mapping words (str) to their rank/index (int). Only set after fit_on_texts was called.

* **document_count:** int. Number of documents (texts/sequences) the tokenizer was trained on. Only set after fit_on_texts or fit_on_sequences was called.



In [80]:
print(len(t.word_counts))
t.word_counts

27873


OrderedDict([('soldier', 3582),
             ('shot', 10784),
             ('parliament', 6992),
             ('locked', 211),
             ('down', 11448),
             ('after', 53169),
             ('gunfire', 1968),
             ('erupts', 93),
             ('at', 97886),
             ('war', 9703),
             ('memorial', 3741),
             ('tourist', 170),
             ('dubbed', 594),
             ('‘spider', 30),
             ('man’', 49),
             ('spider', 6319),
             ('burrows', 248),
             ('under', 12945),
             ('skin', 4315),
             ('for', 147709),
             ('days', 10649),
             ('luke', 1194),
             ('somers', 2614),
             ("'killed", 124),
             ('in', 408093),
             ('failed', 2110),
             ('rescue', 2027),
             ('attempt', 1892),
             ("yemen'", 70),
             ('breaking', 1674),
             ('ottawa', 5095),
             ('giant', 727),
             ('8ft', 76),


In [81]:
t.word_docs

defaultdict(int,
            {'at': 37745,
             'war': 6241,
             'locked': 180,
             'memorial': 1782,
             'shot': 5780,
             'gunfire': 1560,
             'down': 9402,
             'erupts': 93,
             'parliament': 2405,
             'soldier': 2352,
             'after': 28779,
             'tourist': 146,
             '‘spider': 30,
             'dubbed': 594,
             'man’': 49,
             'burrows': 248,
             'for': 45015,
             'days': 7777,
             'under': 10200,
             'spider': 1884,
             'skin': 2191,
             "'killed": 124,
             'attempt': 1728,
             'rescue': 1411,
             'somers': 558,
             'in': 60082,
             'luke': 705,
             "yemen'": 70,
             'failed': 1813,
             'breaking': 1646,
             'ottawa': 1995,
             'weighing': 316,
             'ever': 4283,
             'caught': 3669,
             'and': 5

In [82]:
t.word_index

{'the': 1,
 'to': 2,
 'a': 3,
 'of': 4,
 'in': 5,
 'and': 6,
 'that': 7,
 'is': 8,
 'was': 9,
 'on': 10,
 'for': 11,
 'said': 12,
 'he': 13,
 'with': 14,
 'it': 15,
 'his': 16,
 'have': 17,
 'as': 18,
 'by': 19,
 'has': 20,
 'at': 21,
 'from': 22,
 'be': 23,
 'an': 24,
 'not': 25,
 'are': 26,
 'been': 27,
 '”': 28,
 'but': 29,
 'who': 30,
 'this': 31,
 'had': 32,
 'they': 33,
 'after': 34,
 'i': 35,
 'were': 36,
 'we': 37,
 'will': 38,
 'about': 39,
 'one': 40,
 'or': 41,
 'isis': 42,
 'video': 43,
 'which': 44,
 'she': 45,
 'apple': 46,
 'state': 47,
 'up': 48,
 'her': 49,
 'would': 50,
 'their': 51,
 'more': 52,
 'also': 53,
 'when': 54,
 'told': 55,
 'new': 56,
 'out': 57,
 'no': 58,
 'all': 59,
 'people': 60,
 'there': 61,
 'you': 62,
 'its': 63,
 'if': 64,
 'him': 65,
 'islamic': 66,
 'news': 67,
 'man': 68,
 'could': 69,
 'what': 70,
 'year': 71,
 'watch': 72,
 'time': 73,
 'al': 74,
 'over': 75,
 'some': 76,
 'group': 77,
 'according': 78,
 'u': 79,
 's': 80,
 'into': 81,
 'firs

In [83]:
t.document_count

99944

### Now, tokenize the sentences using nltk sent_tokenize() and encode the senteces with the ids we got form the above `t.word_index`

Initialise 2 lists with names `texts` and `articles`.

```
texts = [] to store text of article as it is.

articles = [] split the above text into a list of sentences.
```

In [0]:
texts = list(dataset["articleBody"])
articles = []
for text in texts:
  articles.append(nltk.sent_tokenize(text))

## Check 2:

first element of texts and articles should be as given below. 

In [86]:
texts[0]

'A small meteorite crashed into a wooded area in Nicaragua\'s capital of Managua overnight, the government said Sunday. Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city\'s airport, the Associated Press reports. \n\nGovernment spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth." House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports. \nMurillo said Nicaragua will ask international experts to help local scientists in understanding what happened.\n\nThe crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee. He said it is still not clear if the meteorite disintegrated or was buried.\n\nHumbe

In [87]:
articles[0]

["A small meteorite crashed into a wooded area in Nicaragua's capital of Managua overnight, the government said Sunday.",
 "Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city's airport, the Associated Press reports.",
 'Government spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth."',
 'House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports.',
 'Murillo said Nicaragua will ask international experts to help local scientists in understanding what happened.',
 'The crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee.',
 'He said it is still not clear if the meteorite disintegrated or was bu

# Now iterate through each article and each sentence to encode the words into ids using t.word_index  [5 marks] 

Here, to get words from sentence you can use `text_to_word_sequence` from keras preprocessing text.

1. Import text_to_word_sequence

2. Initialize a variable of shape (no.of articles, MAX_SENTS, MAX_SENT_LENGTH) with name `data` with zeros first (you can use numpy [np.zeros](https://docs.scipy.org/doc/numpy/reference/generated/numpy.zeros.html) to initialize with all zeros)and then update it while iterating through the words and sentences in each article.

In [0]:
from keras.preprocessing.text import text_to_word_sequence

In [0]:
data = np.zeros((len(texts),MAX_SENTS, MAX_SENT_LENGTH))

In [107]:
data[0, :, :]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.

In [0]:
i = 0
for article in articles:
  j = 0
  for sentence in article:
    if j < MAX_SENTS:
      word_seq = text_to_word_sequence(sentence)
      k = 0
      for word in word_seq:
        if k < MAX_SENT_LENGTH:
          data[i,j,k]=t.word_index[word]
        k += 1
    j += 1
  i += 1

### Check 3:

Accessing first element in data should give something like given below.

In [109]:
data[0,:,:]

array([[3.0000e+00, 4.8100e+02, 4.2700e+02, 7.2110e+03, 8.1000e+01,
        3.0000e+00, 3.7340e+03, 3.3100e+02, 5.0000e+00, 3.8920e+03,
        3.5000e+02, 4.0000e+00, 1.4310e+03, 2.9600e+03, 1.0000e+00,
        8.9000e+01, 1.2000e+01, 4.6600e+02, 0.0000e+00, 0.0000e+00],
       [7.5800e+02, 9.5000e+01, 1.0470e+03, 3.0000e+00, 2.6790e+03,
        1.7520e+03, 7.0000e+00, 1.8900e+02, 3.0000e+00, 1.2170e+03,
        1.0750e+03, 2.0300e+03, 7.0000e+02, 1.5900e+02, 1.0000e+00,
        3.0330e+03, 4.4800e+02, 1.0000e+00, 5.5500e+02, 2.3500e+02],
       [8.9000e+01, 1.0680e+03, 4.1170e+03, 2.3490e+03, 1.2000e+01,
        3.0000e+00, 1.0920e+03, 3.3070e+03, 1.9000e+01, 1.0000e+00,
        8.9000e+01, 2.0000e+00, 1.7930e+03, 1.0000e+00, 5.2100e+02,
        2.0090e+03, 1.5000e+01, 9.0000e+00, 3.0000e+00, 3.1110e+03],
       [1.8100e+02, 3.6410e+03, 9.7200e+02, 2.0000e+02, 2.5580e+03,
        4.4000e+01, 6.7760e+03, 1.7220e+03, 1.2520e+03, 5.0000e+00,
        1.3324e+04, 1.7943e+04, 1.0000e+00, 7

# Repeat the same process for the `Headings` as well. Use variables with names `texts_heading` and `articles_heading` accordingly. [5 marks] 

In [0]:
texts_heading = []
articles_heading = []

for line in dataset['Headline']:
  texts_heading.append(line)
  articles_heading.append(nltk.sent_tokenize(line))

In [0]:
data_heading = np.zeros((len(texts_heading),MAX_SENTS_HEADING, MAX_SENT_LENGTH))

In [126]:
data_heading[0,:,:]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.]])

In [0]:
i = 0
for article_hd in articles_heading:
  j = 0
  for sentence_hd in article_hd:
    if j < MAX_SENTS_HEADING:
      word_seq = text_to_word_sequence(sentence_hd)
      k = 0
      for word in word_seq:
        if k < MAX_SENT_LENGTH:
          data_heading[i,j,k]=t.word_index[word]
        k += 1
    j += 1
  i += 1

In [128]:
data_heading[0,:,:]

array([[  717.,   206.,   343.,  7118.,   193.,    34.,  1338., 11495.,
           21.,   233.,   686.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.]])

### Now the features are ready, lets make the labels ready for the model to process.

### Convert labels into one-hot vectors

You can use [get_dummies](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html) in pandas to create one-hot vectors.

In [0]:
labels = np.array(pd.get_dummies(dataset['Stance']))

In [130]:
labels[:50]

array([[0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [1, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 1, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0,

### Check 4:

The shape of data and labels shoould match the given below numbers.

In [131]:
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (49972, 20, 20)
Shape of label tensor: (49972, 4)


### Shuffle the data

In [0]:
## get numbers upto no.of articles
indices = np.arange(data.shape[0])
## shuffle the numbers
np.random.shuffle(indices)

In [0]:
## shuffle the data
data = data[indices]
data_heading = data_heading[indices]
## shuffle the labels according to data
labels = labels[indices]

### Split into train and validation sets. Split the train set 80:20 ratio to get the train and validation sets.


Use the variable names as given below:

x_train, x_val - for body of articles.

x-heading_train, x_heading_val - for heading of articles.

y_train - for training labels.

y_val - for validation labels.



In [0]:
from sklearn.model_selection import train_test_split
x_train, x_val,x_heading_train, x_heading_val,y_train,y_val = train_test_split(data,data_headings,labels,test_size=0.2)

### Check 5:

The shape of x_train, x_val, y_train and y_val should match the below numbers.

In [136]:
print(x_train.shape)
print(y_train.shape)

print(x_val.shape)
print(y_val.shape)

(39977, 20, 20)
(39977, 4)
(9995, 20, 20)
(9995, 4)


### Create embedding matrix with the glove embeddings


Run the below code to create embedding_matrix which has all the words and their glove embedding if present in glove word list.

In [138]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('./glove.6B.100d.txt')
for line in f:
	  values = line.split()
	  word = values[0]
	  coefs = np.asarray(values[1:], dtype='float32')
	  embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

# create a weight matrix for words in training docs
vocab_size = len(t.word_counts)
embedding_matrix = np.zeros((vocab_size, 100))


for word, i in t.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

Loaded 400000 word vectors.


# Try the sequential model approach and report the accuracy score. [10 marks]  

In [139]:
print('Shape of x_train:', x_train.shape)
print('Shape of x_val:', x_val.shape)
print('Shape of x_heading_train:', x_heading_train.shape)
print('Shape of x_heading_val:', x_heading_val.shape)
print('Shape of y_train:', y_train.shape)
print('Shape of y_val:', y_val.shape)

Shape of x_train: (39977, 20, 20)
Shape of x_val: (9995, 20, 20)
Shape of x_heading_train: (39977, 1, 20)
Shape of x_heading_val: (9995, 1, 20)
Shape of y_train: (39977, 4)
Shape of y_val: (9995, 4)


In [0]:
x_train_full = np.append(x_train,x_heading_train,axis=1)

In [0]:
train_x = x_train_full.reshape(x_train_full.shape[0],x_train_full.shape[1]*x_train_full.shape[2])

In [144]:
train_x.shape

(39977, 420)

In [0]:
x_val_full = np.append(x_val,x_heading_val,axis=1)

In [0]:
val_x = x_val_full.reshape(x_val_full.shape[0],x_val_full.shape[1]*x_val_full.shape[2])

In [145]:
val_x.shape

(9995, 420)

### Import layers from Keras to build the model

In [0]:
from keras.models import Model,Sequential
from keras.initializers import Constant
from keras.layers import Dense, LSTM, Embedding, Flatten, Dropout, SpatialDropout1D, BatchNormalization, Input, concatenate,Activation,Bidirectional, GlobalMaxPool1D, merge
from keras.callbacks import EarlyStopping
from keras.optimizers import SGD, Adam

## **Model 1**


In [0]:
embeding_dimensions = 100

In [0]:
model_1 = Sequential()
model_1.add(Embedding(vocab_size, embeding_dimensions, embeddings_initializer=Constant(embedding_matrix), input_length=420, trainable=False))
model_1.add(SpatialDropout1D(0.3))
model_1.add(LSTM(100, dropout=0.1, recurrent_dropout=0.1))
model_1.add(Dense(50,activation='relu'))
model_1.add(Dropout(0.3))
model_1.add(Dense(4,activation='softmax'))

### Compile and fit the model

In [0]:
model_1.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])

In [180]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
model_1.fit(train_x, y_train, epochs = 5, batch_size=64, verbose = 1, validation_data=(val_x,y_val), callbacks=[es])

Train on 39977 samples, validate on 9995 samples
Epoch 1/5
39977/39977 [==============================] - 468s 12ms/step - loss: 0.8234 - acc: 0.7293 - val_loss: 0.8049 - val_acc: 0.7292
Epoch 2/5
39977/39977 [==============================] - 466s 12ms/step - loss: 0.8064 - acc: 0.7318 - val_loss: 0.8032 - val_acc: 0.7292
Epoch 3/5
39977/39977 [==============================] - 466s 12ms/step - loss: 0.8048 - acc: 0.7318 - val_loss: 0.8041 - val_acc: 0.7292
Epoch 4/5
39977/39977 [==============================] - 465s 12ms/step - loss: 0.8032 - acc: 0.7318 - val_loss: 0.8048 - val_acc: 0.7292
Epoch 5/5
39977/39977 [==============================] - 463s 12ms/step - loss: 0.8024 - acc: 0.7318 - val_loss: 0.8042 - val_acc: 0.7292
Epoch 00005: early stopping


## Build the same model with attention layers included for better performance (Optional)

## Fit the model and report the accuracy score for the model with attention layer (Optional)